In [1]:
pwd

'/home/ubuntu/astromer/presentation/notebooks'

In [2]:
cd ../..

/home/ubuntu/astromer


In [3]:
# Import required modules
from src.data.record import DataPipeline
import tensorflow as tf

import pandas as pd
import numpy as np
from os.path import join
import polars as pl
import os
from glob import glob
import sys
# sys.path.append('/home/Samsung2TB/')
# sys.path.append('/home/Samsung2TB/astromer/')
from tqdm import tqdm
import os
from joblib import Parallel, delayed
from src.data.record import DataPipeline, write_config, deserialize
import tensorflow as tf

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from time import time
import logging 
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)

In [4]:
# Define local path
local_path = '/home/ubuntu/astromer/data/raw_data/alcock/'

# Define Metadata and LCDIR paths
METAPATH = join(local_path, 'new_metadata.parquet')
LCDIR =  join(local_path, 'parquets/')

In [5]:
# Read metadata
metadata = pd.read_parquet(METAPATH)
metadata


,ID,Class,Path,Band,newID,Label
0,1.3319.10,LPV,1.3319.10.dat,1.0,0,3
1,1.3441.15,Cep_0,1.3441.15.dat,1.0,1,0
2,1.3441.25,LPV,1.3441.25.dat,1.0,2,3
3,1.3441.45,Cep_0,1.3441.45.dat,1.0,3,0
4,1.3441.1031,RRab,1.3441.1031.dat,1.0,4,4
...,...,...,...,...,...,...
21439,9.5608.870,RRab,9.5608.870.dat,1.0,21439,4
21440,9.5608.946,RRab,9.5608.946.dat,1.0,21440,4
21441,9.5609.22,EC,9.5609.22.dat,1.0,21441,2
21442,9.5609.790,RRab,9.5609.790.dat,1.0,21442,4


In [6]:
# Process metadata
metadata['Class'] = pd.Categorical(metadata['Class'])
metadata['Label'] = metadata['Class'].cat.codes
metadata['Path'] = metadata['Path'].apply(lambda x: os.path.join(LCDIR, x)) 

In [7]:
# Sample the metadata
metadata_sample = metadata.copy()

In [8]:
metadata_sample

,ID,Class,Path,Band,newID,Label
0,1.3319.10,LPV,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,0,3
1,1.3441.15,Cep_0,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,1,0
2,1.3441.25,LPV,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,2,3
3,1.3441.45,Cep_0,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,3,0
4,1.3441.1031,RRab,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,4,4
...,...,...,...,...,...,...
21439,9.5608.870,RRab,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,21439,4
21440,9.5608.946,RRab,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,21440,4
21441,9.5609.22,EC,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,21441,2
21442,9.5609.790,RRab,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,21442,4


In [9]:
context_features = [ "ID", "Label", "Class"]
sequential_features = [ "mjd", "mag"]
write_config(context_features = context_features, sequential_features = sequential_features)

2023-06-28 11:28:44,121 - INFO - Successfully wrote config to ./config.toml


In [10]:
# Create an instance of DataPipeline
myPipeline = DataPipeline(metadata=metadata_sample)


[INFO] 21444 samples loaded


In [11]:

# Prepare for k-fold cross-validation
test_metadata = metadata_sample.sample(n=1000)
k_folds = 2
# Split the data into training, validation, and test sets
myPipeline.train_val_test(val_frac=0.2, 
                        test_meta=[test_metadata], 
                        k_fold=k_folds)


[INFO] Using ID col as sample identifier
[INFO] Shuffling
[INFO] Shuffling


In [12]:
# Create TFRecord files and measure time taken
a = time()
var = myPipeline.run(LCDIR, METAPATH, n_jobs=2,)
b = time()
print(b-a)

2023-06-28 11:28:47,002 - INFO - Starting DataPipeline operations
Writting test fold 0:   0%|          | 0/2 [00:00<?, ?it/s]    2023-06-28 11:28:47,042 - INFO - wrote tfrecords to ./records/output/test_0_2.record
2023-06-28 11:28:47,042 - INFO - wrote tfrecords to ./records/output/test_0_3.record
2023-06-28 11:28:47,042 - INFO - wrote tfrecords to ./records/output/test_0_4.record
2023-06-28 11:28:47,043 - INFO - wrote tfrecords to ./records/output/test_0_5.record
Writting train fold 0:   0%|          | 0/2 [00:00<?, ?it/s]    2023-06-28 11:28:47,050 - INFO - wrote tfrecords to ./records/output/train_0_2.record
2023-06-28 11:28:47,051 - INFO - wrote tfrecords to ./records/output/train_0_3.record
2023-06-28 11:28:47,051 - INFO - wrote tfrecords to ./records/output/train_0_4.record
2023-06-28 11:28:47,052 - INFO - wrote tfrecords to ./records/output/train_0_5.record
Writting validation fold 0:   0%|          | 0/2 [00:00<?, ?it/s]    2023-06-28 11:28:47,058 - INFO - wrote tfrecords to ./

0.08318376541137695


In [13]:
import glob

record_files = glob.glob('/home/ubuntu/astromer/records/output/*.record')
raw_dataset = tf.data.TFRecordDataset(record_files)


2023-06-28 11:28:48.420208: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-28 11:28:48.428135: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-28 11:28:48.428773: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-28 11:28:48.429835: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [15]:
myPipeline.inspect_records()

2023-06-28 11:28:56,907 - INFO - Successfully inspected 1 records from ./records/output/train_0.record.


features {
  feature {
    key: "Class"
    value {
      bytes_list {
        value: "EC"
      }
    }
  }
  feature {
    key: "ID"
    value {
      bytes_list {
        value: "2.4903.2543"
      }
    }
  }
  feature {
    key: "Label"
    value {
      int64_list {
        value: 2
      }
    }
  }
}



In [16]:
def deserialize_dataset(raw_dataset, show_progress=True):
    """
    Function to deserialize a TFRecord dataset.

    Args:
        raw_dataset (tf.data.TFRecordDataset): The raw TFRecord dataset.
        show_progress (bool): If True, display a progress bar.

    Returns:
        list: List of deserialized samples.
    """

    # Get the number of records in the dataset (needed for the progress bar)
    num_records = sum(1 for _ in raw_dataset)

    # Initialize the tqdm progress bar if show_progress is True
    pbar = tqdm(total=num_records, ncols=70, disable=not show_progress, desc='Deserializing records')

    # The list to store the deserialized data
    processed_data = []

    # Deserialize each record in the dataset
    for raw_record in raw_dataset:
        # Parse the raw record
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        
        # Deserialize the record and append it to the list
        processed_data.append(deserialize(example.SerializeToString()))
        
        # Update the progress bar
        pbar.update()

    # Close the progress bar
    pbar.close()

    # Return the list of deserialized data
    return processed_data


In [17]:
processed_data = deserialize_dataset(raw_dataset)

Deserializing records: 100%|█| 107220/107220 [02:42<00:00, 660.47it/s]


In [18]:
processed_data

[{'ID': <tf.Tensor: shape=(), dtype=string, numpy=b'82.8410.1233'>,
  'Label': <tf.Tensor: shape=(), dtype=int64, numpy=4>,
  'Class': <tf.Tensor: shape=(), dtype=string, numpy=b'RRab'>,
  'input': <tf.Tensor: shape=(0, 0, 2), dtype=float32, numpy=array([], shape=(0, 0, 2), dtype=float32)>},
 {'ID': <tf.Tensor: shape=(), dtype=string, numpy=b'15.10675.278'>,
  'Label': <tf.Tensor: shape=(), dtype=int64, numpy=2>,
  'Class': <tf.Tensor: shape=(), dtype=string, numpy=b'EC'>,
  'input': <tf.Tensor: shape=(0, 0, 2), dtype=float32, numpy=array([], shape=(0, 0, 2), dtype=float32)>},
 {'ID': <tf.Tensor: shape=(), dtype=string, numpy=b'76.10572.31'>,
  'Label': <tf.Tensor: shape=(), dtype=int64, numpy=0>,
  'Class': <tf.Tensor: shape=(), dtype=string, numpy=b'Cep_0'>,
  'input': <tf.Tensor: shape=(0, 0, 2), dtype=float32, numpy=array([], shape=(0, 0, 2), dtype=float32)>},
 {'ID': <tf.Tensor: shape=(), dtype=string, numpy=b'15.10918.20'>,
  'Label': <tf.Tensor: shape=(), dtype=int64, numpy=3>,
